### Using Keras with TensorFlow backend

In [1]:
import numpy as np

from keras.models import Model
from keras.layers import Flatten, Dense, Input, Activation
from keras.layers import BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing import image
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.models import Sequential
import h5py

Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

### Set GPU
tf.device('/gpu:0') tells TF to use gpu 0. Can use gpu 1. If cell is not run then fills both GPUs

In [3]:
import tensorflow as tf

In [2]:
#val = np.loadtxt('kaggle/val.txt',dtype='str',delimiter=' ')

In [3]:
#Grab image paths put in list
#path = 'kaggle/train_sm/'
#img_paths = []
#for i in val:
#    img_paths.append(path + i[0])
#print len(img_paths)

688


### Grab Test Image file paths and put into List

In [2]:
import os

In [3]:
#Grab image paths put in list
path = 'UCMerced/Images/'
img_paths = []
label = []
for i in sorted(os.listdir(path)):
    for j in sorted(os.listdir(path+i)):
        img_paths.append(path + i + '/'+ j)
        label.append(j)
print len(img_paths)

2100


In [4]:
new_path = 'Cifar/Val/'
#Grab image paths put in list
img_paths = []
for i in sorted(os.listdir(new_path)):
    small_path = new_path+i+'/'
    for j in sorted(os.listdir(small_path)):
        img_paths.append(small_path + j)
print len(img_paths)

10000


In [58]:
#val = np.loadtxt('kaggle/val.txt',dtype='str',delimiter=' ')

In [64]:
#Grab Training image paths put in list
#path = 'kaggle/train_sm/'
#img_paths = []
#labels = []
#for i in val:
#    img_paths.append(path + i[0])
#    labels.append(i[1])
#print len(img_paths)

688


### Import Functional Keras model from keras.applications Library 

In [5]:
#from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
#from myresnet50 import ResNet50

### Create Instance of base model 
include_top = False gets rid of top model.

weights = 'imagenet' Sets weights to image net weights.

In [6]:
#model = ResNet50(include_top=False, weights='imagenet', input_tensor=Input(shape=(224,224,3)))
#with tf.device('/gpu:1'):
model = VGG16(include_top=True, weights='imagenet', input_tensor=Input(shape=(224,224,3)))


### Build Top Model

In [8]:
#Top of the model
x = model.layers[18].output
x = Flatten(name='flatten')(x)

#x = Dense(4096, name='fc1002',init='lecun_uniform')(x)
#x = BatchNormalization(axis=1,name='batch_norm2')(x)
#x = Activation("relu",name='bfc1002')(x)

x = Dense(4096, name='fc1001',init='lecun_uniform')(x) 
x = BatchNormalization(axis=1, name='batch_norm')(x)
x = Activation("relu",name='bfc1001')(x)

# 344 is the number of classes
x = Dense(344, name='fc1000',init='lecun_uniform')(x)
x = Activation("softmax",name='bfc1000')(x)


### Create a New Model Class where input is base model and output is Top model

In [9]:
# Stack top_model on top
final_model = Model(input=model.input, output=x)

In [10]:
#File handle to open weights path
#weights_path = 'Caffe models/ResNet50/weights/ResNet50-finetune5-14-0.66.h5'
weights_path = 'Caffe models/VGG/VGG16-finetune8less-44-0.22.h5'
f = h5py.File(weights_path)

In [14]:
#K.get_session().run(tf.global_variables_initializer())

### For each layer set the weights from the .h5 file

In [11]:
for k in range(len(f.attrs['layer_names'])): 
    name = f.attrs['layer_names'][k]
    weights = [f[name][h][()] for h in f[name].keys()]
    #Skip over setting Batch Normalization weightt. 176 is the BN layer in the top model
    #if "BatchNormalization" not in str(final_model.layers[k]) or k == 176:
    print k
    final_model.layers[k].set_weights(weights)
f.close()
print('Model loaded.')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
Model loaded.


### Create a new model to feature extract and any desired layer

In [16]:
layer_name = 'fc1001'
intermediate_layer_model = Model(input=final_model.input,
                                 output=final_model.get_layer(layer_name).output)
#intermediate_layer_model = Model(input=model.input,
#                                 output=model.get_layer('avg_pool').output)

In [17]:
def preprocess_input(x):
    x[:, :, :, 0] -= 103.939
    x[:, :, :, 1] -= 116.779
    x[:, :, :, 2] -= 123.68
    # 'RGB'->'BGR'
    x = x[:, :, :, ::-1]
    x = x*(1./255) #comment out for orignal ResNet model. Affects numbers
    return x

In [18]:
#Test for 1 image 
img = image.load_img(img_paths[0], target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
test = intermediate_layer_model.predict(x)

In [20]:
#test

In [ ]:
#Get the first weight of each layer to check how big by sending image through
for name in f.attrs['layer_names']:
    try:
        model_b.get_layer(name).output
    except:
        continue
    intermediate_layer_model = Model(input=model_b.input,
                                     output=model_b.get_layer(name).output)
    img = image.load_img(img_paths[0], target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    test = intermediate_layer_model.predict(x)
    print name
    print test[0].flatten()[0]


In [21]:
features = []
for i in range(len(img_paths)):
    img = image.load_img(img_paths[i], target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features.append(intermediate_layer_model.predict(x).flatten())

In [22]:
image_features = np.squeeze(features)

In [23]:
image_features.shape

(2100, 4096)

### Process to Save extracted Features into Dataframe then to CSV

In [24]:
import pandas as pd

In [25]:
df1 = pd.DataFrame(np.array(label))
df2 = pd.DataFrame(image_features)

In [27]:
frames = [df1, df2]
result = pd.concat(frames,axis=1)

In [28]:
result.head()

,0,0,1,2,3,4,5,6,7,8,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,agricultural00.tif,-8.144235,-9.100907,0.242139,0.136577,0.078200,-0.314120,-3.557962,-0.968221,1.337975,...,-0.004864,-4.771656,-9.397353,-2.721172,-11.032389,5.551828,10.028757,6.802113,8.201835,-4.912762
1,agricultural01.tif,-5.526604,-7.298079,0.662029,0.496180,0.896402,-2.030433,-2.464485,0.172785,0.029954,...,0.046032,-3.789477,-9.630476,-4.182361,-10.135643,4.088016,9.197919,5.611478,8.728189,-2.870421
2,agricultural02.tif,-5.425224,-5.934672,-0.320346,1.241558,0.974716,-0.517567,-3.624277,1.233659,1.128209,...,0.708996,-2.528896,-9.955143,-3.576707,-10.006432,5.623075,10.368506,4.985160,8.789730,-4.683414
3,agricultural03.tif,-5.734372,-6.086385,0.078721,0.935184,0.756439,-0.734438,-3.422276,-0.556330,1.080915,...,-0.319884,-3.432272,-8.958613,-3.897922,-9.524585,4.696538,8.964489,6.794896,7.747476,-3.959482
4,agricultural04.tif,-15.973546,-13.659833,-0.959912,-7.951974,0.854418,5.629112,-9.034302,-2.414236,-1.916469,...,-6.482616,-7.786484,-15.660973,2.489746,-16.015741,2.337044,11.651434,7.406279,12.399714,-0.463293


In [29]:
result.to_csv('Features Extracted/UCMerced/Finetune/Single/UCM-VGG16-44epoch-fc1001-22-fine8less.csv',index=False)

In [47]:
df = pd.read_csv('Features Extracted/UCMerced/NoFinetune/UCM-ResNet50-avgpool.csv')
df.head()

,0,0.1,1,2,3,4,5,6,7,8,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,agricultural00.tif,0.909985,0.030925,0.087651,0.0,0.058850,0.00000,0.222092,0.040623,0.066892,...,0.000000,0.292268,0.323953,0.003515,0.091544,0.136804,0.071006,0.027468,0.016721,0.059630
1,agricultural01.tif,1.107137,0.092292,0.126290,0.0,0.116014,0.00000,0.000000,0.135269,0.000795,...,0.000000,0.094603,0.000000,0.258073,0.278475,0.000000,0.159937,0.016492,0.013055,0.039381
2,agricultural02.tif,0.705839,0.068317,0.045845,0.0,0.296381,0.08295,0.863768,0.000000,0.000000,...,0.000000,0.232503,0.000000,0.000000,0.229589,0.112848,0.162782,0.077588,0.000000,0.146023
3,agricultural03.tif,0.843741,0.000000,0.016553,0.0,0.107277,0.00000,0.194399,0.008352,0.000000,...,0.000000,0.034639,0.000000,0.146941,0.377052,0.000000,0.008070,0.000000,0.000000,0.056305
4,agricultural04.tif,0.006460,0.163063,0.048054,0.0,0.020554,0.00000,0.000000,0.000000,0.000000,...,0.035747,0.000000,0.000000,0.120376,0.000000,0.000000,0.000000,0.627859,0.000000,0.002354


### Test if weights before/after finetuning change

In [8]:
model_k = ResNet50(include_top=False, weights='imagenet', input_tensor=Input(shape=(224,224,3)))
#model_b = ResNet50(include_top=False, weights='imagenet', input_tensor=Input(shape=(224,224,3)))

In [9]:
from collections import defaultdict

In [10]:
FLayers = defaultdict(bool)
for k in range(175):
    #f = h5py.File(weights_path)
    #name = f.attrs['layer_names'][k]
    #weights = [f[name][h][()] for h in f[name].keys()]
    #final_model_fine.layers[k].set_weights(weights)

    orig = model_k.layers[k].get_weights()
    fine = model_b.layers[k].get_weights()

    params = 0
    for o,f in zip(orig,fine):
        a = (o == f)
        a = a.flatten()
        params += sum(a)

    FLayers[k] = params == model_k.layers[k].count_params()
#f.close()

In [11]:
uF = []
for key,value in FLayers.items():
    if value == False:
        uF.append(key)

In [12]:
test = 0
for u in uF:
    print model_k.layers[u]